In [5]:
!pip install transformers tqdm more_itertools scikit-learn torch

In [ ]:
# Use this when working on the full entity dataset of 260_000 entities
# mention names aren't ordered inn a particular way, just what appears first in the documents
# we can have many duplicates, so in this case where we are only encoding the name we want to avoid that
# unique_mention_name_id_pairs = list({name: _id for name, _id in bc5cdr_name_id_pairs}.items())
# mention_names = unique_mention_name_id_pairs.keys()

# entity names are sorted alphabetically, so we'll need to randomly sample to not just get names starting with a
# import random
# entity_names_ids = [(name, id) for (name, id) in mesh_name_id_pairs]
# entity_names = [name for (name, entity) in entity_names_ids]



In [ ]:
# Core SapBert model for outputing dense vector representations
# this is the feature extraction pipeline so we can get the embeddings directly (we can only do inference with this, no fine-tuning)
from transformers import pipeline

model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"

# core model
extractor = pipeline("feature-extraction", model=model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
from datasets import load_dataset

# there are all "positive" pairs"
dataset = load_dataset("Stevenf232/BC5CDR_MeSH2015_nameonly")
train_pairs = dataset["train"]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

name_only_train.jsonl: 0.00B [00:00, ?B/s]

name_only_val.jsonl: 0.00B [00:00, ?B/s]

name_only_test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2656 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm
def extract_features(extractor ,pairs):
  batch_size=16
  # input all names to model so it will create dense vectors of all the names (of the mentions and entities)
  #mention_name_features = extractor(mention_names, batch=16, truncation=True, padding=True)
  mention_name_features = []
  entity_name_features = []

  for i in tqdm(range(0, len(pairs), batch_size), desc="Extracting features"):
      # extract mention features
      batch = pairs[i:i + batch_size]["mention"]
      features = extractor(batch, truncation=True, padding=True)
      mention_name_features.extend(features)

      # extract entity features
      batch = pairs[i:i + batch_size]["entity"]
      features = extractor(batch, truncation=True, padding=True)
      entity_name_features.extend(features)

  return mention_name_features, entity_name_features

In [ ]:
mention_name_features, entity_name_features = extract_features(extractor, train_pairs)

Extracting features: 100%|██████████| 166/166 [01:31<00:00,  1.82it/s]


In [ ]:
# ------------------ evaluating core model -------------------------
# once we have our features in shape [n,m,i]
# The CLS token which represents the word is in [0,0,:]

import numpy as np
mention_vectors = [np.array(f[0][0]) for f in mention_name_features]
mention_vectors[0].shape


# entity name vectors
# entity_name_context_vectors = [np.array(feature) for feature in entity_name_features]
# entity_vectors = [context_vector[0,0,:] for context_vector in entity_name_context_vectors]

# another method
# entity_name_context_vectors = np.array(entity_name_features)
# entity_vectors = np.array([context_vector[0,0,:] for context_vector in entity_name_context_vectors])

entity_vectors = [np.array(f[0][0]) for f in entity_name_features]
entity_vectors[0].shape



NameError: name 'mention_name_features' is not defined

In [ ]:
mention_vectors[0][:5]

array([-0.31827238,  0.12176217, -0.26363215, -0.07197451, -0.04306259])

# Simple model evaluation

Potential issue - this finds relevance using Cosine Similarity (will it have bias towards fine-tuning on cosineSimilarityLoss vs other loss functions?)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate(mention_vectors, entity_vectors):
  correct_count = 0
  for i, vector in enumerate(mention_vectors):
      scores = cosine_similarity(vector.reshape(1,-1), entity_vectors).flatten().tolist()
      top_idx = np.argmax(scores)
      # the strange conversion to int from here on out is because the original idx is of type numpy.int64
      top_match_id = train_pairs[int(top_idx)]["id"]
      correct_id = train_pairs[int(i)]["id"]

      if top_match_id == correct_id:
          correct_count += 1

      mention_name = train_pairs[int(i)]["mention"]
      top_match = train_pairs[int(top_idx)]["entity"]
      correct_name = train_pairs[int(i)]["entity"]
      print(f"mention_name: {mention_name}\ncorrect entity name: {correct_name}\ntop_match: {top_match}\n")


  print(f"total comparisons: {len(mention_vectors)}")
  print(f"correct comparisons: {correct_count}")
  print(f"accuracy: {correct_count / len(mention_vectors)}")

In [ ]:
evaluate(mention_vectors, entity_vectors)

Streaming output truncated to the last 5000 lines.

mention_name: calcium chloride
correct entity name: Calcium Chloride
top_match: Calcium Chloride

mention_name: neoplasms
correct entity name: Neoplasms
top_match: Neoplasms

mention_name: streptococcal endophthalmitis
correct entity name: Streptococcal Infections
top_match: Endophthalmitis

mention_name: cranial hemorrhage
correct entity name: Cerebral Hemorrhage
top_match: Intracranial Hemorrhages

mention_name: depressive symptoms
correct entity name: Depressive Disorder
top_match: Depressive Disorder

mention_name: hepatorenal syndrome
correct entity name: Hepatorenal Syndrome
top_match: Hepatorenal Syndrome

mention_name: Migraine
correct entity name: Migraine Disorders
top_match: Migraine Disorders

mention_name: Alopecia
correct entity name: Alopecia
top_match: Alopecia

mention_name: hypoxia
correct entity name: Anoxia
top_match: Hypoxia, Brain

mention_name: thyroxine
correct entity name: Thyroxine
top_match: Thyroxine

menti

In [ ]:
# more evaluation methods
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# print(f"{accuracy_score(train_labels, predicted_labels)=:.3f}")
# print(f"{recall_score(train_labels, predicted_labels)=:.3f}")
# print(f"{precision_score(train_labels, predicted_labels)=:.3f}")
# print(f"{f1_score(train_labels, predicted_labels)=:.3f}")

## Evaluating Fine-tuned model

In [1]:
# finetuned model
# don't think I need a feature-extraction pipeline based on auto-generated docs: https://huggingface.co/Stevenf232/fine-tuned-SapBERT2
#from transformers import pipeline

from sentence_transformers import SentenceTransformer

fine_tuned_model_name = "Stevenf232/fine-tuned-SapBERT4"
model = SentenceTransformer(fine_tuned_model_name)

# fine_tuned_extractor = pipeline("feature-extraction", model=fine_tuned_model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

In [2]:
from tqdm import tqdm
def encode(pairs):
  batch_size=16
  mention_encodings = []
  entity_encodings = []

  for i in tqdm(range(0, len(pairs), batch_size), desc="Extracting features"):
      # encode mentions
      batch = pairs[i:i + batch_size]["mention"]
      encodings = model.encode(batch)
      mention_encodings.extend(encodings)

      # encode entities
      batch = pairs[i:i + batch_size]["entity"]
      encodings = model.encode(batch)
      entity_encodings.extend(encodings)

  return mention_encodings, entity_encodings

In [7]:
# mention_name_features, entity_name_features = extract_features(fine_tuned_extractor, train_pairs)
mention_encodings, entity_encodings = encode(train_pairs)

Extracting features: 100%|██████████| 166/166 [03:34<00:00,  1.29s/it]


In [ ]:
# ------------------ evaluating fine-tuned model -------------------------
# don't think I actually need to do this because we already have the pooling layer implemented as part of the sentece-transformer
# mention_vectors = [np.array(f[0][0]) for f in mention_name_features]
# entity_vectors = [np.array(f[0][0]) for f in entity_name_features]

In [ ]:
mention_vectors[0][:5]

In [11]:
evaluate(mention_encodings, entity_encodings)

Streaming output truncated to the last 5000 lines.

mention_name: calcium chloride
correct entity name: Calcium Chloride
top_match: Calcium Chloride

mention_name: neoplasms
correct entity name: Neoplasms
top_match: Neoplasms

mention_name: streptococcal endophthalmitis
correct entity name: Streptococcal Infections
top_match: Streptococcal Infections

mention_name: cranial hemorrhage
correct entity name: Cerebral Hemorrhage
top_match: Intracranial Hemorrhages

mention_name: depressive symptoms
correct entity name: Depressive Disorder
top_match: Depressive Disorder

mention_name: hepatorenal syndrome
correct entity name: Hepatorenal Syndrome
top_match: Hepatorenal Syndrome

mention_name: Migraine
correct entity name: Migraine Disorders
top_match: Migraine Disorders

mention_name: Alopecia
correct entity name: Alopecia
top_match: Alopecia

mention_name: hypoxia
correct entity name: Anoxia
top_match: Anoxia

mention_name: thyroxine
correct entity name: Thyroxine
top_match: Thyroxine

ment